In [1]:
# Imports
import numpy as np
import random as rand
import tensorflow as tf
import pandas as pd
import keras
import matplotlib
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Add as keras_add_layers
from keras.callbacks import History
np.set_printoptions(suppress=True)

Using TensorFlow backend.


In [6]:
# Static letter variable
letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
sequence_length = 10

# Generates a random sequence of letters
def generate_random_sequence_of_letters(length=sequence_length):
    sequence = ''
    for _ in range(length):
        sequence += letters[rand.randint(0,len(letters) - 1)]
    return sequence

# Shifts the letters in a sequence by the specified amount
def shift_sequence(sequence,shift_size=rand.randint(0,len(letters) - 2)):
    ret_sequence = ''
    for letter in sequence:
        index = letters.index(letter)
        index += shift_size
        while index > len(letters) - 1:
            index -= len(letters)
        ret_sequence += letters[index]
    return ret_sequence
            
# Converts a sequence of letters into an index vector
def convert_to_index_vector(sequence):
    ret_vector = []
    for letter in sequence:
        ret_vector.append(float(letters.index(letter)))
    return ret_vector

# Generates the scrambled inputs for a given sequence
def generate_scrambled_inputs(sequence):
    input_vector = convert_to_index_vector(shift_sequence(sequence,shift_size=12))
    return input_vector

# Converts a sequence to a hot vector sequence
def convert_sequence_to_hot_vectors(sequence):
    vectors = []
    for num in sequence:
        vector = np.zeros(len(letters))
        vector[int(num)] = 1
        vectors.append(vector)
    return np.array(vectors)

# Converts a hot vector sequence to a letter sequence
def convert_hot_vectors_to_sequence(vectors):
    indices = [np.argmax(vector) for vector in vectors]
    return convert_index_vector(indices)

# Generates training data
def generate_training_data(size=10000):
    inputs = []
    outputs = []
    for _ in range(size):
        sequence = generate_random_sequence_of_letters()
        outputs.append(convert_sequence_to_hot_vectors(convert_to_index_vector(sequence)))
        inputs.append(convert_sequence_to_hot_vectors(generate_scrambled_inputs(sequence)))
    return [np.array(inputs),np.array(outputs)]

# Converts an index vector to a string
def convert_index_vector(index_vector):
    sequence = ''
    for index in index_vector:
        if round(index) < len(letters):
            sequence += letters[int(round(index))]
        else:
            sequence += '*'
    return sequence

In [12]:
# Create model
model = Sequential()
model.add(LSTM(len(letters),input_shape=(sequence_length,len(letters)),return_sequences=True))
model.add(Dense(len(letters),activation='softmax'))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['acc'])

In [10]:
# Actual generation of training_data
training_data = generate_training_data(10000)
training_inputs = training_data[0]
training_outputs = training_data[1]

In [13]:
# Training of the model
history = model.fit(training_inputs,training_outputs,batch_size=256,epochs=100,validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 3s 394us/step - loss: 0.1616 - acc: 0.9615 - val_loss: 0.1603 - val_acc: 0.9615
Epoch 2/100
8000/8000 [==============================] - 2s 210us/step - loss: 0.1590 - acc: 0.9615 - val_loss: 0.1573 - val_acc: 0.9615
Epoch 3/100
8000/8000 [==============================] - 2s 209us/step - loss: 0.1552 - acc: 0.9615 - val_loss: 0.1520 - val_acc: 0.9615
Epoch 4/100
8000/8000 [==============================] - 2s 234us/step - loss: 0.1468 - acc: 0.9615 - val_loss: 0.1380 - val_acc: 0.9615
Epoch 5/100
8000/8000 [==============================] - 2s 207us/step - loss: 0.1270 - acc: 0.9615 - val_loss: 0.1142 - val_acc: 0.9615
Epoch 6/100
8000/8000 [==============================] - 2s 229us/step - loss: 0.1025 - acc: 0.9615 - val_loss: 0.0892 - val_acc: 0.9616
Epoch 7/100
8000/8000 [==============================] - 2s 210us/step - loss: 0.0776 - acc: 0.9623 - val_loss: 0.0649 - val_acc: 

Epoch 59/100
8000/8000 [==============================] - 2s 251us/step - loss: 2.5618e-04 - acc: 1.0000 - val_loss: 2.5076e-04 - val_acc: 1.0000
Epoch 60/100
6912/8000 [========================>.....] - ETA: 0s - loss: 2.4627e-04 - acc: 1.0000

KeyboardInterrupt: 

In [7]:
# Generate test data
test_data = generate_training_data(100)
test_inputs = test_data[0]
test_outputs = test_data[1]

In [8]:
# Print predictions vs actual
outputs = model.predict(test_inputs)
for i in range(len(test_outputs)):
    print(convert_hot_vectors_to_sequence(outputs[i]),convert_hot_vectors_to_sequence(test_outputs[i]))
    #print(convert_index_vector(outputs[i]),convert_index_vector(test_outputs[i]))

ValueError: Error when checking input: expected lstm_1_input to have shape (10, 26) but got array with shape (20, 26)